In [4]:
import pandas as pd

# Analisis de trayectorio

Dataset de la informacion de paraderos (busstop.csv):
- Muestra informacion de los paraderos que se utilizan durante el trayectorio de los autobuses, tanto como la latitud y longitud.
- Se consideran puntos gps en formato de latitud y longitud


Se procesa la nueva trayectoria de paraderos y analizar los radios que se les asignaran para el geofencing, creando un nuevo archivo de la trayectoria de paraderos.

In [5]:
# cargar datos en bruto
busstop_df = pd.read_csv('../data/busstops_ida.csv', sep=',')
busstop_df

,number,id,name,navigation,latitude,longitude,radio
0,0,0,inicio,159,-13.554978,-71.856025,30
1,1,36,primer stop,304,-13.554812,-71.857300,18
2,2,37,segundo stop,281,-13.553294,-71.861944,13
3,3,38,Ccollana,283,-13.552296,-71.867299,9
4,4,39,Puente Huaccoto,283,-13.551592,-71.870211,10
...,...,...,...,...,...,...,...
79,79,114,Segundo,221,-13.547528,-71.985248,10
80,80,115,Reservorio,225,-13.549660,-71.988003,10
81,81,116,Cuarto,230,-13.550408,-71.988880,10
82,82,117,Quinto,203,-13.550841,-71.989448,10


# Analisis de los registros de monitorizacion

In [3]:
# cargar datos en bruto
monitoringdf = pd.read_csv('../data/_Process_monitoring__202303251137.csv', sep=',')

# Creacion del las columnas datetime y latitude_longitude
monitoringdf['datetime'] = monitoringdf['date'] + ' ' + monitoringdf['time']
monitoringdf['latitude_longitude'] = monitoringdf.apply(lambda row: (row['latitude'], row['longitude']), axis=1)

# eliminar las columnas latitude, longitude, time, velocity. Luego reordenar columnas
monitoringdf.drop(columns=['latitude', 'longitude','time'], inplace=True)
monitoringdf = monitoringdf[['id', 'date','datetime', 'latitude_longitude', 'lap', 'navigation', 'vehicle_id']]
monitoringdf

,id,date,datetime,latitude_longitude,lap,navigation,vehicle_id
0,13173398,2023-03-01,2023-03-01 05:10:11,"(-13.5495973, -71.8775099)",1,285,SJ39
1,13177467,2023-03-01,2023-03-01 06:04:51,"(-13.5245292, -71.9775272)",1,298,SJ34
2,13189674,2023-03-01,2023-03-01 07:22:21,"(-13.5341828, -71.9807088)",1,7,SJ29
3,13192729,2023-03-01,2023-03-01 07:39:04,"(-13.5511464, -71.9869284)",1,202,SJ21
4,13193272,2023-03-01,2023-03-01 07:41:52,"(-13.548642, -71.88101)",1,107,SJ34
...,...,...,...,...,...,...,...
1685065,17241845,2023-03-25,2023-03-25 08:50:24,"(-13.5254398, -71.9750786)",2,288,SJ24
1685066,17253753,2023-03-25,2023-03-25 10:01:56,"(-13.5511209, -71.9896213)",2,253,SJ16
1685067,17257644,2023-03-25,2023-03-25 10:26:10,"(-13.5432069, -71.8977847)",2,126,SJ24
1685068,17266468,2023-03-25,2023-03-25 11:14:26,"(-13.5327737, -71.9218652)",1,258,SJ43
